In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Data loading
train_dataframe = pandas.read_csv('train.csv')

In [3]:
train_dataframe.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
cols =['Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']
for c in cols:
    print('%s: %.2e' % (c, train_dataframe[c].isnull().sum()/train_dataframe[c].size))

Pclass: 0.00e+00
Sex: 0.00e+00
Age: 1.99e-01
SibSp: 0.00e+00
Parch: 0.00e+00
Ticket: 0.00e+00
Fare: 0.00e+00
Cabin: 7.71e-01
Embarked: 2.24e-03


In [5]:
import re
train_dataframe['Title'] = train_dataframe.Name.apply(lambda x: re.search(' ([A-Z][a-z]+)\.', x).group(1))

Title_Dictionary = {"Capt": "Officer",
                        "Col": "Officer",
                        "Major": "Officer",
                        "Dr": "Officer",
                        "Rev": "Officer",
                        "Jonkheer": "Royalty",
                        "Don": "Royalty",
                        "Sir" : "Royalty",
                        "the Countess": "Royalty",
                        "Dona": "Royalty",
                        "Lady": "Royalty",
                        "Mme": "Mrs",
                        "Ms": "Mrs",
                        "Mrs": "Mrs",
                        "Mlle": "Miss",
                        "Miss": "Miss",
                        "Mr": "Mr",
                        "Master": "Master"}
    
train_dataframe['Title'] = train_dataframe.Title.map(Title_Dictionary) 
embarked_group = train_dataframe.groupby(['Sex','Pclass','Title'])['Embarked']
print(embarked_group.apply(lambda x: x.mode()[0]))

Sex     Pclass  Title  
female  1       Miss       C
                Mrs        S
                Officer    S
                Royalty    C
        2       Miss       S
                Mrs        S
        3       Miss       S
                Mrs        S
male    1       Master     S
                Mr         S
                Officer    S
                Royalty    C
        2       Master     S
                Mr         S
                Officer    S
        3       Master     S
                Mr         S
Name: Embarked, dtype: object


In [6]:
def prepr(dataframe):
    # Not needed: 'PassengerId', 'Name'
    # Not yet treated: 'Ticket', 'Cabin'
    # Features: 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'
    # Labels: 'Survived'
        
    # need to find Na for Age
    import re
    dataframe['Title'] = dataframe.Name.apply(lambda x: re.search(' ([A-Z][a-z]+)\.', x).group(1))

    Title_Dictionary = {"Capt": "Officer",
                        "Col": "Officer",
                        "Major": "Officer",
                        "Dr": "Officer",
                        "Rev": "Officer",
                        "Jonkheer": "Royalty",
                        "Don": "Royalty",
                        "Sir" : "Royalty",
                        "the Countess": "Royalty",
                        "Dona": "Royalty",
                        "Lady": "Royalty",
                        "Mme": "Mrs",
                        "Ms": "Mrs",
                        "Mrs": "Mrs",
                        "Mlle": "Miss",
                        "Miss": "Miss",
                        "Mr": "Mr",
                        "Master": "Master"}
    
    dataframe['Title'] = dataframe.Title.map(Title_Dictionary)
    group = dataframe.groupby(['Sex','Pclass','Title'])
    
    # missing Age -> median
    dataframe.loc[dataframe.Age.isnull(), 'Age'] = group.Age.transform('median')
    #dataframe.loc[dataframe.Age.isnull(), 'Age'] = group.Age.apply(lambda x: x.median())
    # print(group.Age.apply(lambda x: x.median()))
    # print(group.Age.transform('median'))
    
    # missing Embarked -> mode
    #dataframe.loc[dataframe.Embarked.isnull(), 'Embarked'] = group.Embarked.apply(lambda x: x.mode())
    
    # Sex
    mp = {'male': 1, 'female': -1}
    dataframe['Sex'] = dataframe['Sex'].map(lambda x : mp.get(x) if x in mp else x)
    
    # Embarked
    mp = {'C': -1, 'S': 0, 'Q': 1}
    dataframe['Embarked'] = dataframe['Embarked'].map(lambda x : mp.get(x) if x in mp else x)
    
    # treat Na as 0
    dataframe.fillna(0,inplace=True)
    
    return dataframe[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

In [7]:
X_train_orig = prepr(train_dataframe).as_matrix()
Y_train = train_dataframe['Survived'].as_matrix().ravel()

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_orig)
X_train = scaler.transform(X_train_orig)

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split

param_grid = [{'kernel': ['rbf'], 'C': [10, 100, 1000], 'gamma' : [1e-2, 1e-3, 1e-4]},
              {'kernel': ['linear'], 'C': [1, 10, 100]}]

X_train_a, X_train_v, Y_train_a, Y_train_v = train_test_split(X_train, Y_train, random_state=42)
gcv = GridSearchCV(SVC(), param_grid, cv=5, n_jobs=4)
gcv.fit(X_train_a, Y_train_a)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid=[{'kernel': ['rbf'], 'C': [10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]}, {'kernel': ['linear'], 'C': [1, 10, 100]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [10]:
mu = gcv.cv_results_['mean_test_score']
std = gcv.cv_results_['std_test_score']
pars = gcv.cv_results_['params']
for (m,s,p) in zip(mu,std,pars):
    print('%.2f (+-%.2f): %s' %(m,s,p))

0.80 (+-0.04): {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
0.79 (+-0.04): {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.72 (+-0.02): {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.81 (+-0.03): {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
0.79 (+-0.04): {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.79 (+-0.04): {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.81 (+-0.03): {'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
0.80 (+-0.04): {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.79 (+-0.04): {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.79 (+-0.04): {'C': 1, 'kernel': 'linear'}
0.79 (+-0.04): {'C': 10, 'kernel': 'linear'}
0.79 (+-0.04): {'C': 100, 'kernel': 'linear'}


In [11]:
pred = gcv.predict(X_train_v)

from sklearn.metrics import classification_report, accuracy_score
print('hyperparams: %s' % gcv.best_params_)
print(classification_report(pred,Y_train_v))
print('accuracy %.2f' % accuracy_score(pred,Y_train_v))

hyperparams: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
             precision    recall  f1-score   support

          0       0.89      0.85      0.87       140
          1       0.76      0.82      0.79        83

avg / total       0.84      0.84      0.84       223

accuracy 0.84
